<h3>Importing Data Cleaning and Merging it.</h3>

In [5]:
import pandas as pd

# Load the data
people_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\01_people.csv")
abilities_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\02_abilities.csv")
education_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\03_education.csv")
skills_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\06_skills.csv")
experience_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\04_experience.csv")
person_skill_df = pd.read_csv(r"C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\54k Resume dataset (structured)\05_person_skills.csv")



<h3>Abilities CSV</h3>
<p>The Data is Structured like multiple repeated rows with skills of individual People. People are identified by Person_id(primary Key)
we will be keeping only first 5000 people for our project the dataset has like 50000+ people with thier data in it</p>

In [32]:
abilities_aggregated = abilities_df.groupby('person_id')['ability'].apply(lambda x: ', '.join(x)).reset_index()

# Save the aggregated data to a new CSV
abilities_aggregated = abilities_aggregated.head(4999)
abilities_aggregated.to_csv('02_abilities.csv', index=False)

<h3>People CSV</h3>
<p>we will be removing the unnecsary columns from this csv which are email, linkdein and phone which are at average 90% null and we dont need them in our project</p>

In [ ]:
columns_to_drop = ['email', 'phone', 'linkedin']
df_cleaned = people_df.drop(columns=columns_to_drop)

# Save the cleaned data to a new CSV file
df_cleaned = df_cleaned.head(5000)
df_cleaned.to_csv('01_people.csv', index=False)


print("Columns removed and cleaned file saved as '01_people_cleaned.csv'")

Columns removed and cleaned file saved as '01_people_cleaned.csv'


<h3>Education CSV</h3>
<p>There are 5 columns person_id, institution, program, start_date and location, we will be removing start_date and Location since we dont need them. we will also be joining multiple person_id's rows.</p>

In [34]:

# Merge the education data into one row per person_id, combining all institutions and programs
df_education_merged = education_df.groupby('person_id').agg({
    'institution': lambda x: ', '.join(x.dropna().unique()),  # Concatenate unique institutions
    'program': lambda x: ', '.join(x.dropna().unique()),      # Concatenate unique programs
}).reset_index()

df_education_merged = df_education_merged.head(5000)
# Save the cleaned and merged data to a new CSV file
df_education_merged.to_csv('03_education.csv', index=False)


print("Education data has been merged and cleaned.")


Education data has been merged and cleaned.


<h3>Experience CSV</h3>
<p>So this one needs more work than any other. we will first merge multiple person_id rows into 1, than we will calculate the years a person worked for, if missing we will add zero instead. </p>

In [ ]:
import pandas as pd
from datetime import datetime

# Ensure that 'title' and 'firm' columns are treated as strings
experience_df['title'] = experience_df['title'].fillna('').astype(str)
experience_df['firm'] = experience_df['firm'].fillna('').astype(str)

# Convert start_date and end_date to datetime, coercing any errors
experience_df['start_date'] = pd.to_datetime(experience_df['start_date'], errors='coerce')
experience_df['end_date'] = pd.to_datetime(experience_df['end_date'], errors='coerce')

# Fill missing end_date values with the current date
experience_df['end_date'].fillna(datetime.now(), inplace=True)

# Calculate the duration in years (difference between start and end date)
experience_df['duration_years'] = (experience_df['end_date'] - experience_df['start_date']).dt.days / 365.25

# Group by 'person_id' and aggregate data
df_experience_grouped = experience_df.groupby('person_id').agg({
    'title': ' '.join,  # Concatenate job titles for each person
    'firm': ' '.join,   # Concatenate firm names for each person
    'start_date': 'min',  # Keep earliest start date
    'end_date': 'max',    # Keep latest end date
    'duration_years': 'sum'  # Sum the durations for all jobs
}).reset_index()

# Save the cleaned and grouped experience data to a new CSV file
df_experience_grouped = df_experience_grouped.head(5000)
df_experience_grouped.to_csv('04_experience_grouped.csv', index=False)



In [ ]:
import pandas as pd
from datetime import datetime

# Load your 04_experience dataset
df = pd.read_csv('C:/Users/DELL/Desktop/Projects/DataScience/AI-powered-Resume-Builder-and-Reviewer/data/Preprocessing/04_experience_cleaned.csv')

# Fill missing end dates with today's date for entries with 0.0 duration
current_date = datetime.today().strftime('%Y-%m-%d')

# Replace empty or None End Dates with today's date
df['end_date'].fillna(current_date, inplace=True)

# Ensure the columns 'Start Date' and 'End Date' are in datetime format (removing time part)
df['start_date'] = pd.to_datetime(df['start_date']).dt.date
df['end_date'] = pd.to_datetime(df['end_date']).dt.date

# Recalculate the duration for those entries that have missing or invalid end dates
df['Calculated Duration'] = (pd.to_datetime(df['end_date']) - pd.to_datetime(df['start_date'])).dt.days / 365

# Optionally, update the duration for rows with original 0.0 duration values
df.loc[df['Calculated Duration'] == 0.0, 'Calculated Duration'] = (pd.to_datetime(df['end_date']) - pd.to_datetime(df['start_date'])).dt.days / 365

# Handle NaN or infinite values in the 'Calculated Duration' column
df['Calculated Duration'].fillna(0, inplace=True)  # Replace NaN with 0 (or another default value)
df['Calculated Duration'] = df['Calculated Duration'].replace([float('inf'), -float('inf')], 0)  # Replace infinite values with 0

# Round the 'Calculated Duration' to the nearest whole number (years) and convert to integer
df['Calculated Duration'] = df['Calculated Duration'].round().astype(int)

# Drop the 'Start Date', 'End Date', and any other columns related to time
df.drop(columns=['start_date', 'end_date', 'duration_years'], inplace=True)

# Save the updated DataFrame back to a CSV file if necessary
df.to_csv('C:/Users/DELL/Desktop/Projects/DataScience/AI-powered-Resume-Builder-and-Reviewer/data/Preprocessing/04_experience.csv', index=False)


<h3>Person Skills CSV</h3>
<p>So this one needs more work than any other. we will first merge multiple person_id rows into 1, than we will calculate the years a person worked for, if missing we will add zero instead. </p>

In [9]:
person_skill_df['skill'] = person_skill_df['skill'].fillna('').astype(str)

# Group by 'person_id' and concatenate 'skills' for each person
person_skills_grouped = person_skill_df.groupby('person_id')['skill'].apply(lambda x: ', '.join(x)).reset_index()

# Save the merged DataFrame back to a CSV file
person_skills_grouped.to_csv('C:/Users/DELL/Desktop/Projects/DataScience/AI-powered-Resume-Builder-and-Reviewer/data/Preprocessing/05_person_skills.csv', index=False)

# Display the grouped DataFrame
print(person_skills_grouped)

       person_id                                              skill
0              1  Database administration, Database, Ms sql serv...
1              2  sql server management studio, visual studio, s...
2              3  DATABASES, ORACLE (4 years), ORACLE 10G, SQL, ...
3              4  Maintain multiple database environments (Redsh...
4              5  Scrum, Agile software development, Product bac...
...          ...                                                ...
54853      54929  Django, Angular JS, JavaScript, JQuery, Node.j...
54854      54930  Python, Django, AWS, AngularJS, Bootstrap, Jav...
54855      54931  Python 2.7, HTML5, CSS3, AJAX, JSON, JQuery, A...
54856      54932  Python 3.1x, PyQuery, PyQt, Django, Angular.js...
54857      54933  Analyzed and gathered business requirements, D...

[54858 rows x 2 columns]


In [14]:
import pandas as pd

# Load all the cleaned CSV files
people_df = pd.read_csv(r'C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\Preprocessing\01_people.csv')
person_skills_df = pd.read_csv(r'C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\Preprocessing\05_person_skills.csv')
education_df = pd.read_csv(r'C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\Preprocessing\03_education.csv')
experience_df = pd.read_csv(r'C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\Preprocessing\04_experience.csv')
abilities_df = pd.read_csv(r'C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\Preprocessing\02_abilities.csv')

# Perform inner join on 'person_id' column to merge all data
merged_df = people_df.merge(person_skills_df, on='person_id', how='inner')
merged_df = merged_df.merge(education_df, on='person_id', how='inner')
merged_df = merged_df.merge(experience_df, on='person_id', how='inner')
merged_df = merged_df.merge(abilities_df, on='person_id', how='inner')

# Save the final merged DataFrame to a new CSV file (provide the full file path with .csv extension)
merged_df.to_csv(r'C:\Users\DELL\Desktop\Projects\DataScience\AI-powered-Resume-Builder-and-Reviewer\data\Preprocessing\final_merged_data.csv', index=False)

# Display the merged DataFrame
print(merged_df)


      person_id                                               name  \
0             1                             Database Administrator   
1             2                             Database Administrator   
2             3                      Oracle Database Administrator   
3             4  Amazon Redshift Administrator and ETL Develope...   
4             5             Scrum Master Scrum Master Scrum Master   
...         ...                                                ...   
4489       4994                               Front- End developer   
4490       4996                                   Sr. UI Developer   
4491       4998                                Freelance Developer   
4492       4999                         Senior Front End Developer   
4493       5000                      DEVELOPER DEVELOPER DEVELOPER   

                                                  skill  \
0     Database administration, Database, Ms sql serv...   
1     sql server management studio, visua

In [22]:
df= pd.read_csv(r'final_merged_data.csv')
df.isnull().sum().sum()
df.fillna('UNKOWN')


,person_id,name,skill,institution,program,title,firm,Calculated Duration,ability
0,1,Database Administrator,"Database administration, Database, Ms sql serv...",Lead City University,Bachelor of Science,Database Administrator Database Administrator,Family Private Care LLC Incomm,11,"Installation and Building Server, Running Back..."
1,2,Database Administrator,"sql server management studio, visual studio, s...",lagos state university,bsc in computer science,Database Administrator,Intercontinental Registry,3,"database management systems administration, de..."
2,3,Oracle Database Administrator,"DATABASES, ORACLE (4 years), ORACLE 10G, SQL, ...","JNTU - Kakinada, Andhra Pradesh",Master of Computer Applications in Science and...,Oracle Database Administrator Oracle Database ...,Cognizant Convergys,11,"Over 4+ years of Experience as Architecture, E..."
3,4,Amazon Redshift Administrator and ETL Develope...,Maintain multiple database environments (Redsh...,University of Informatics,Bachelor in Computer Science,Amazon Redshift Administrator and ETL Develope...,"MSP Recovery - Fort Lauderdale, FL CEAACES - Q...",0,"SQL management, PostgresSQL, Oracle, MySQL, mi..."
4,5,Scrum Master Scrum Master Scrum Master,"Scrum, Agile software development, Product bac...",Virginia Commomwealth University,UNKOWN,Scrum Master Oracle Database Administrator/ Sc...,Quest Technologies Prudential Time Warner Cable,7,"Scrum Master, Agile software development, Prod..."
...,...,...,...,...,...,...,...,...,...
4489,4994,Front- End developer,"ASP.NET 4.6/4.5, Web Services, Angular 2/4/5, ...",DePaul University,Master of Science in Information Systems,Front- End developer Net developer/ Front- End...,Bank Of America NV Energy United Airlines Citi,22,"SDLC Agile/ Scrum Development, Waterfall, ASP...."
4490,4996,Sr. UI Developer,"CSS LESS/SASS preprocessors, CSS Backgrounds, ...",Florida State University,Master's Degree in Computer Science,Sr. UI Developer Sr. UI developer Sr. UI Devel...,"JP Morgan Chase Miller Publishing Group, LLC A...",18,"HTML, JavaScript, AJAX, CSS, jQuery"
4491,4998,Freelance Developer,"Wordpress, WooCommerce, CSS/SASS, Javascript, ...",Full Sail University,Associate in Digital Media,Freelance Developer Front-End Developer Senior...,Freelance Everbank/TIAA Bank Moxie ShopVisible...,17,Semantic and Accessible HTML using W3C and WCA...
4492,4999,Senior Front End Developer,"Spring, Hibernate, Struts, JQuery, Ajax, Sench...",STATE UNIVERSITY,MASTER OF COMPUTER SCIENCE,Senior Front End Developer Senior Front End De...,TeamSoft Inc FireEye AppLabs Pyramid Technolog...,0,"Requirements Analysis, Design, Development, Im..."
